### Import Modules and Packages

This tutorial requires the use of one built-in Python module and a number of 3rd party ones. I'll quickly run through why we need each:

* `random`: Helps to generate random points in Part 2 of the tutorial. A Python built-in module.
* `geopandas`: For operations on GeoDataFrames
* `json`: For plotting maps with Plotly
* `matplotlib.pyplot`: For plotting figures and maps
* `numpy`: Contains a number of statistical functions
* `plotly.express`: For interactive plotting
* `pandas`: For operations on DataFrames
* `tobler`: Contains the areal interpolation functions. Part of the PySal library.
* `scipy`: Contains a number of statistical functions
* `shapely.geometry`: For the `Point` class which is used when generating the random points

If you need help on importing modules then this guide from Real Python should help: [Python Modules and Packages – An Introduction](https://realpython.com/python-modules-packages/)

In [ ]:
# Import modules
import random

import geopandas as gpd
import json
import matplotlib.pyplot as plt # Do I need this if all the plots are with gpd.explore() or px?
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import tobler
from scipy import stats
from shapely.geometry import Point # Maybe I should import the whole module?

---
## Part 1 - Areal Interpolation of Census Population Data

### Read The Data

We first need read the data from our `data/` directory and assign them to variables.

In [ ]:
# Read the City of Ottawa census tracts data
ct_gdf = gpd.read_file('data/ottawa_ct_pop_2016.gpkg')

# Read the City of Ottawa dissemination areas data
da_gdf = gpd.read_file('data/ottawa_da_pop_2016.gpkg')

# Read the City of Ottawa neighborhoods data
nbhd_gdf = gpd.read_file('data/ottawa_neighborhoods.gpkg')

### Inspect The Data

The following methods from Pandas and GeoPandas are great ways to inspect DataFrames and GeoDataFrames. Note: The DataFrame methods also work on GeoDataFrames but `explore()` only works with GeoDataFrames

* [`df.info()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.info.html): Information about the DataFrame/GeoDataFrame
* [`df.head()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.info.html): First n rows of the DataFrame/GeoDataFrame (defaults to 10 rows)
* [`df.tail()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.tail.html): Last n rows of the DataFrame/GeoDataFrame (defaults to 10 rows) 
* [`df.sample()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.sample.html): Random sample of n rows of the DataFrame/GeoDataFrame (defaults to 10 rows) 
* [`gdf.explore()`](https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoDataFrame.explore.html): Interactive map of a GeoDataFrame
* [`gdf.crs`](https://geopandas.org/en/stable/docs/user_guide/projections.html): Coordinate reference system information of a GeoDataFrame

Just replace `df` or `gdf` before the dot notation with your DataFrame or GeoDataFrame's name

### Interpolation

ct_gdf['ct_pop_2016'] = ct_synth_gdf['ct_synth_pop']
nbhd_gdf['nbhd_pop_est'] = nbhd_synth_gdf['nbhd_synth_pop']

#### Area Weighted Interpolation

- [tobler.area_weighted.area_interpolate](https://pysal.org/tobler/generated/tobler.area_weighted.area_interpolate.html#tobler.area_weighted.area_interpolate)

In [ ]:
# Area weighted interpolation: census tracts to neighborhoods
# -----------------------------------------------------------

ct_area_interp_gdf = tobler.area_weighted.area_interpolate(source_df=ct_gdf, 
                                                        target_df=nbhd_gdf,
                                                        extensive_variables=['ct_pop_2016'])

# Round the interpolation results to the nearest integer and change the type to integer
# (Population counts must be integers)
ct_area_interp_gdf['ct_pop_2016'] = ct_area_interp_gdf['ct_pop_2016'].round(decimals=0).astype(int)

# Rename the results column for clarity later
ct_area_interp_gdf = ct_area_interp_gdf.rename({'ct_pop_2016':'ct_area_interp_est'}, axis=1)

In [ ]:
# Area weighted interpolation: dissemination areas to neighborhoods
# -----------------------------------------------------------------

da_area_interp_gdf = tobler.area_weighted.area_interpolate(source_df=da_gdf, 
                                                        target_df=nbhd_gdf,
                                                        extensive_variables=['da_pop_2016'])

# Round the interpolation results to the nearest integer and change the type to integer
# (Population counts must be integers)
da_area_interp_gdf['da_pop_2016'] = da_area_interp_gdf['da_pop_2016'].round(decimals=0).astype(int)

# Rename the results column for clarity later
da_area_interp_gdf = da_area_interp_gdf.rename({'da_pop_2016':'da_area_interp_est'}, axis=1)

### Dasymetric Interpolation

- [tobler.dasymetric.masked_area_interpolate](https://pysal.org/tobler/generated/tobler.dasymetric.masked_area_interpolate.html#tobler.dasymetric.masked_area_interpolate)

In [ ]:
# Dasymetric interpolation: census tracts + urban landover to neighborhoods
#--------------------------------------------------------------------------

# Perform dasymetric interpolation
ct_dasy_interp_gdf = tobler.dasymetric.masked_area_interpolate(source_df=ct_gdf, 
                                            target_df=nbhd_gdf,
                                            raster='data/ottawa_landcover.tif',
                                            codes=[17],
                                            extensive_variables=['ct_pop_2016'])

# Round the interpolation results to the nearest integer and change the type to integer
# (Population counts must be integers)
ct_dasy_interp_gdf['ct_pop_2016'] = ct_dasy_interp_gdf['ct_pop_2016'].round(decimals=0).astype(int)

# Rename the results column for clarity later
ct_dasy_interp_gdf = ct_dasy_interp_gdf.rename({'ct_pop_2016':'ct_dasy_interp_est'}, axis=1)

# ~30 s ; ignore warnings

In [ ]:
# Dasymetric interpolation: dissemination areas + urban landover to neighborhoods
#------------------------------------------------------------------------

# Perform dasymetric interpolation
da_dasy_interp_gdf = tobler.dasymetric.masked_area_interpolate(source_df=da_gdf, 
                                                            target_df=nbhd_gdf,
                                                            raster='data/ottawa_landcover.tif',
                                                            codes=[17],
                                                            extensive_variables=['da_pop_2016'])

# Round the interpolation results to the nearest integer and change the type to integer
# (Population counts must be integers)
da_dasy_interp_gdf['da_pop_2016'] = da_dasy_interp_gdf['da_pop_2016'].round(decimals=0).astype(int)

# Rename the results column for clarity later
da_dasy_interp_gdf = da_dasy_interp_gdf.rename({'da_pop_2016':'da_dasy_interp_est'}, axis=1)

# ~1.5 mins : ignore warnings

### Assessing the Interpolation Results

Because the neighborhoods data came with a population estimate for each neighborhood (`nbhd_pop_est`) we can use it to compare those values against the interpolated values. It is very important to note that these assessments are very limited in their accuracy for a number of reasons:

1. Metadata associated with the Ottawa neighborhood survey data does not specify the year of the population estimate. This data was first published to the Open Ottawa data portal on November 25, 2019 and then updated on June 2, 2021. The census tract population data comes from the 2016 census.
2. Metadata associated with the Ottawa neighborhood survey data does not specify the methodology they used for estimating the neighborhood populations. Perhaps it came from an areal interpolation method similar to what we are using or perhaps it came from a higher resolution data set that the City of Ottawa has but which they don't publish (e.g. size of each households).
3. The Ottawa neighborhood survey data indicates that the 'Beechwood Cemetery' and 'Notre-Dame Cemetery' neighborhoods have populations of 139 and 17, respectively, despite no evidence of residences within them. 
4. The sum of all the neighborhood population estimates is 867146 while the census_tracts' sum is 934243.

Despite this we will go ahead and use these estimates to assess the interpolation results. We will do the following:

1. Visual assessment of the percent error of each neighborhood (interpolated population vs `nbhd_pop_est`)
2. Checking the population sums
3. Statistical assessment of the Root Mean Square Error (RMSE) and Mean Bias Error (MBE) of each neighborhood (interpolated population vs `nbhd_pop_est`)

In [ ]:
# Merge the results for comparison
#---------------------------------

# Create a list of the GeoDataFrames (drop the redundant geometry)
dfs = [nbhd_gdf,
        ct_area_interp_gdf.drop(columns='geometry'),
        ct_dasy_interp_gdf.drop(columns='geometry'),
        da_area_interp_gdf.drop(columns='geometry'),
        da_dasy_interp_gdf.drop(columns='geometry'),]

# Concatenate the GeoDataFrames
interp_results_gdf = pd.concat(dfs, axis=1)

# Get into on the new interpolation results GeoDataFrame
interp_results_gdf.info()

In [ ]:
# Define functions to assess the results
# --------------------------------------

# PE
def percent_error(estimated, expected):
    ''' Return Percent Error where estimated and expected are numeric or array-like'''
    return abs(((estimated - expected) / expected) * 100)

# RMSE
def rmse(estimated, expected):
    ''' Return Root Mean Square Error where estimated and expected are array-like'''
    return np.sqrt(np.mean((estimated - expected) ** 2))

# NRMSE
def nrmse(estimated, expected):
    ''' Return Normalized Root Mean Square Error where estimated and expected are array-like'''
    return np.sqrt(np.mean((estimated - expected) ** 2))/np.std(estimated)

# MBE
def mbe(estimated, expected):
    ''' Return Mean Bias Error where estimated and expected are array-like'''
    return np.mean(estimated - expected)

# MAE
def mae(estimated, expected):
    ''' Return Mean Mean Absolute Error where estimated and expected are array-like'''
    return np.mean(abs(estimated - expected))

# r value
def r_value(estimated, expected):
    ''' Return r value where estimated and expected are array-like.'''
    slope, intercept, r_value, p_value, std_err = stats.linregress(estimated, expected)
    return r_value

In [ ]:
# Evaluate Results
# ----------------

ct_area_est = interp_results_gdf['ct_area_interp_est']
ct_dasy_est = interp_results_gdf['ct_dasy_interp_est']
da_area_est = interp_results_gdf['da_area_interp_est']
da_dasy_est = interp_results_gdf['da_dasy_interp_est']
expected = interp_results_gdf['nbhd_pop_est']

# Percent Error - create new columns in interp_results_gdf
interp_results_gdf['ct_area_%_error'] = round(percent_error(ct_area_est, expected), 1)
interp_results_gdf['ct_dasy_%_error'] = round(percent_error(ct_dasy_est, expected), 1)
interp_results_gdf['da_area_%_error'] = round(percent_error(da_area_est, expected), 1)
interp_results_gdf['da_dasy_%_error'] = round(percent_error(da_dasy_est, expected), 1)

# Other statistics - create DataFrame of statistics
interp_stats_df = pd.DataFrame({'Interpolation Method': ['Area Weighted','Area Weighted',
                                                        'Dasymetric', 'Dasymetric'],
                                'Source Geographies': ['Census Tracts', 'Dissemination Areas',
                                                       'Census Tracts', 'Dissemination Areas'],
                                'RMSE': [round(rmse(ct_area_est, expected), 2),
                                        round(rmse(da_area_est, expected), 2),
                                        round(rmse(ct_dasy_est, expected), 2),
                                        round(rmse(da_dasy_est, expected), 2)],
                                'NRMSE': [round(nrmse(ct_area_est, expected), 2),
                                        round(nrmse(da_area_est, expected), 2),
                                        round(nrmse(ct_dasy_est, expected), 2),
                                        round(nrmse(da_dasy_est, expected), 2)],            
                                'MBE': [round(mbe(ct_area_est, expected), 2),
                                        round(mbe(da_area_est, expected), 2),
                                        round(mbe(ct_dasy_est, expected), 2),
                                        round(mbe(da_dasy_est, expected), 2)],
                                'MAE': [round(mae(ct_area_est, expected), 2),
                                        round(mae(da_area_est, expected), 2),
                                        round(mae(ct_dasy_est, expected), 2),
                                        round(mae(da_dasy_est, expected), 2)],
                                'r': [round(r_value(ct_area_est, expected),2 ),
                                        round(r_value(da_area_est, expected), 2),
                                        round(r_value(ct_dasy_est, expected),2 ),
                                        round(r_value(da_dasy_est, expected), 2)],
                                'r2': [round(r_value(ct_area_est, expected)**2, 2),
                                        round(r_value(da_area_est, expected)**2, 2),
                                        round(r_value(ct_dasy_est, expected)**2, 2),
                                        round(r_value(da_dasy_est, expected)**2, 2)]})

interp_stats_df

In [ ]:
# Visually Assess Percent Error of the Area Weighted Interpolation
# ----------------------------------------------------------------

interp_results_gdf.explore(column='area_%_error',
    cmap='YlOrRd',
    scheme='JenksCaspall',
    legend_kwds={'colorbar':False},
    style_kwds={'weight':1})

In [ ]:
# Visually Assess Percent Error of the Dasymetric Interpolation
# -------------------------------------------------------------

interp_results_gdf.explore(column='dasy_%_error',
    cmap='YlOrRd',
    scheme='JenksCaspall',
    legend_kwds={'colorbar':False},
    style_kwds={'weight':1})

In [ ]:
# Plot the Percent Error of Both Methods using a Plotly Facet Map
# https://plotly.com/python/choropleth-maps/
# https://plotly.github.io/plotly.py-docs/generated/plotly.express.choropleth.html
# https://plotly.com/python/facet-plots/
# --------------------------------------------------------------------------------

# Convert interp_results_gdf from GeoDataFrame to GeoJson
geojson = interp_results_gdf.to_crs(epsg='4326').to_json()
geojson = json.loads(geojson)

# Reformat the interp_results_gdf for the plot
df = pd.DataFrame(interp_results_gdf.drop(columns='geometry'))

df = df.rename({'ct_area_%_error':'Source: Census Tracts <br> Method: Area Weighted',
                'ct_dasy_%_error':'Source: Census Tracts <br> Method: Dasymetric',
                'da_area_%_error':'Source: Dissemination Areas <br> Method: Area Weighted',
                'da_dasy_%_error':'Source: Dissemination Areas <br> Method: Dasymetric',
                }, axis=1)

df = df.melt(id_vars='nbhd_name',
            value_vars=['Source: Census Tracts <br> Method: Area Weighted',
                        'Source: Census Tracts <br> Method: Dasymetric',
                        'Source: Dissemination Areas <br> Method: Area Weighted',
                        'Source: Dissemination Areas <br> Method: Dasymetric'],
            var_name='method', 
            value_name='Error (%)')

# Create the Plotly Express choropleth figure
fig = px.choropleth(data_frame=df,
                    title="Areal Interpolation of 2016 Census Population Data to Ottawa, ON Neighborhoods",
                    locations='nbhd_name',
                    geojson=geojson,
                    featureidkey='properties.nbhd_name',
                    color='Error (%)',
                    facet_col='method',
                    facet_col_wrap=2,
                    range_color=[0,100],
                    color_continuous_scale='Inferno',
                    projection='mercator',
                    fitbounds="locations",
                    height=800, width=800)
         
fig.for_each_annotation(lambda a: a.update(text=a.text.split('=')[-1]))

fig.show()

In [ ]:
# Create 1:1 Facet Plots using Plotly
# -------------------------------------
# https://plotly.com/python/line-and-scatter/
# https://plotly.github.io/plotly.py-docs/generated/plotly.express.scatter.html
# https://plotly.com/python/facet-plots/

# Reformat the interp_results_gdf for the plot
# --------------------------------------------

# Convert the interpolation results to a DataFrame
df = pd.DataFrame(interp_results_gdf.drop(columns='geometry'))

# Rename the results columns as they will become the facet plot labels
df = df.rename({'ct_area_interp_est':'Source: Census Tracts <br> Method: Area Weighted',
                'ct_dasy_interp_est':'Source: Census Tracts <br> Method: Dasymetric',
                'da_area_interp_est':'Source: Dissemination Areas <br> Method: Area Weighted',
                'da_dasy_interp_est':'Source: Dissemination Areas <br> Method: Dasymetric'},
                axis=1)

# Combine all the results columns into one column
df = df.melt(id_vars=['nbhd_name', 'nbhd_pop_est'],
            value_vars=['Source: Census Tracts <br> Method: Area Weighted',
                        'Source: Census Tracts <br> Method: Dasymetric',
                        'Source: Dissemination Areas <br> Method: Area Weighted',
                        'Source: Dissemination Areas <br> Method: Dasymetric'],
            var_name='method', 
            value_name='interp_pop_est')

# Add a percent error column
estimated = df['interp_pop_est']
expected = df['nbhd_pop_est']
df['%_error'] = round(percent_error(estimated, expected), 1)

# Create the Plotly Express Scatter figure
fig = px.scatter(data_frame=df,
                title="Areal Interpolation of 2016 Census Population Data to Ottawa, ON Neighborhoods",
                x='nbhd_pop_est',
                y='interp_pop_est',
                height=800, width=800,
                color='%_error',
                facet_col='method',
                facet_col_wrap=2,
                hover_name='nbhd_name',
                labels={'interp_pop_est': 'Interpolated Population',
                        'nbhd_pop_est': ' Estimated Population'},
                color_continuous_scale='Inferno',
                range_color=[0,100])

# Create the 1:1 line
line_max = max([max(df['interp_pop_est']), max(df['nbhd_pop_est'])])
line_min = min([min(df['interp_pop_est']), min(df['nbhd_pop_est'])])

fig.update_layout(shapes=[
        dict(type='line', xref='x', yref='y',
            x0=line_min, y0=line_min, x1=line_max, y1=line_max, line_width=1),
        dict(type='line', xref='x2', yref='y2',
            x0=line_min, y0=line_min, x1=line_max, y1=line_max, line_width=1),
        dict(type='line', xref='x3', yref='y3',
            x0=line_min, y0=line_min, x1=line_max, y1=line_max, line_width=1),
        dict(type='line', xref='x4', yref='y4',
            x0=line_min, y0=line_min, x1=line_max, y1=line_max, line_width=1)])

fig.for_each_annotation(lambda a: a.update(text=a.text.split('=')[-1]))

fig.show()

---
## Part 2 - Areal Interpolation of Synthetic Population Data

### Generate Synthetic Population Data

- Falls within:
  - Zones that allow residents
  - Urban land cover types
- Population counts based on 2016 census dissemination areas (finer scale than census tracts data)

In [ ]:
# Create clipped dissemination areas
# ----------------------------------

# Read the dissemination areas
da_gdf = gpd.read_file('data/ottawa_da_pop_2016.gpkg')

# Extract the urban landcover (pixel value of 17) from the landcover raster
raster_path = 'data/ottawa_landcover.tif'
urban_landcover_gdf = tobler.dasymetric.extract_raster_features(gdf=nbhd_gdf,
                                                                raster_path=raster_path,
                                                                pixel_values=17)
urban_landcover_gdf = urban_landcover_gdf.to_crs(epsg=32618)

# Read the Ottawa zoning
zoning_gdf = gpd.read_file('data/ottawa_zoning.gpkg')

# Only keep the zones that residents can live in
people_zones = ['R1', 'R2', 'R3', 'R4', 'R5', 'RM', 'LC', 'GM', 'TM',
                'AM','MC','MD', 'AG', 'RR','RU','VM', 'V1','V2','V3']

zoning_gdf = zoning_gdf[zoning_gdf['zone_main'].isin(people_zones)]

# Clip the dissemination areas to the urban landcover and the zoning
da_clip_gdf = gpd.clip(gdf=da_gdf, mask=zoning_gdf)
da_clip_gdf = gpd.clip(gdf=da_clip_gdf, mask=urban_landcover_gdf)

# ~2 mins ; ignore warnings

In [ ]:
# Create points in the clipped dissemination areas
# ------------------------------------------------

def random_points(row, n_column, geom_column, divisor=1, min_n=1):
    ''' Returns n number of random points within GeoDataFrame polygons.

    Parameter:
    ----------
    row : Pandas Series
        Must contain:
        - A column containing the desired number of points in each polygons
        - A column containing polygon geometry

    n_column : string
        - The name of the column that contains the desired number of points

    geom_column : string
        - The name of the column that contains the GeoDataBase geometry

    divisor : int (default = 1)
        - A value to reduce the number of points by that amount
        - Good for reducing computation time

    min_n : int (default = 1)
        - The minimum number of points in polygon
        - Good for forcing all polygons to have at least 1 point
    '''
    
    number = round((row[n_column] / divisor), 0) #/100 == 1.5min; /50=2.5mins; /10=16mins
    if number < min_n:
        number = min_n
    polygon = row[geom_column]
    points = []
    min_x, min_y, max_x, max_y = polygon.bounds
    while len(points) < number:
        point = Point(random.uniform(min_x, max_x), random.uniform(min_y, max_y))
        #if polygon.contains(point):
        if point.intersects(polygon):
            points.append(point)

    return points

In [ ]:
# Generate the random points
synth_points_srs = da_clip_gdf.apply(func=random_points, 
                            args=('da_pop_2016', 'geometry', 10, 1), axis=1)

# The output is a Series so convert it to a list
synth_points_list = list(synth_points_srs)

# Flatten the list of lists so each item in the list is a point
synth_points_flat_list = []
for i in synth_points_list:
    for j in i:
        synth_points_flat_list.append(j)

# Create GeoDataFrame of the synthetic points using the flat list as the geometry column
synth_points_gdf = gpd.GeoDataFrame(geometry=synth_points_flat_list, crs="EPSG:32618")

# ~15-20 mins

In [ ]:
#synth_points_gdf.to_file('data/synth_points_gdf.gpkg', driver='GPKG')

In [ ]:
synth_points_gdf = gpd.read_file("data/synth_points_gdf.gpkg")

In [ ]:
# Determine the census tracts' synthetic populations

# Spatial join: synth_people_gdf ~ census_tracts
ct_points_gdf = gpd.sjoin(synth_points_gdf, ct_gdf, how='inner', predicate='intersects')
ct_points_gdf['ct_synth_pop'] = 1

# Sum the points that fall in each census tract - this is the population of the census tract
ct_points_sums_gdf = ct_points_gdf.groupby(['ctuid']).sum()

# Merge the counts with the census tracts
ct_synth_gdf = ct_gdf.merge(ct_points_sums_gdf[['ct_synth_pop']], on='ctuid')

In [ ]:
# Determine the dissemination areas' synthetic populations

# Spatial join: synth_people_gdf ~ da_gdf
da_points_gdf = gpd.sjoin(synth_points_gdf, da_gdf, how='inner', predicate='intersects')
da_points_gdf['da_synth_pop'] = 1

# Sum the points that fall in each dissemination areas - this is the population of the dissemination area
da_points_sums_gdf = da_points_gdf.groupby(['dauid']).sum()

# Merge the counts with the census tracts
da_synth_gdf = da_gdf.merge(da_points_sums_gdf[['da_synth_pop']], on='dauid')

In [ ]:
# Determine the neighborhoods' synthetic populations

# Spatial join: synthetic points to neighborhoods
nbhd_points_gdf = gpd.sjoin(synth_points_gdf, nbhd_gdf, how='inner', predicate='intersects')
nbhd_points_gdf['nbhd_synth_pop'] = 1

# Sum the points that fall in each neighborhood - this is the population of the neighborhood
nbhd_points_sums_gdf = nbhd_points_gdf.groupby(['nbhd_name']).sum()

# Merge the counts with the neighborhoods
nbhd_synth_gdf = nbhd_gdf.merge(nbhd_points_sums_gdf[['nbhd_synth_pop']], on='nbhd_name')

In [ ]:
# Check the total points in the census tracts, dissemination areas, and neighborhoods

print(ct_synth_gdf['ct_synth_pop'].sum())
print(da_synth_gdf['da_synth_pop'].sum())
print(nbhd_synth_gdf['nbhd_synth_pop'].sum())

In [ ]:
# Area weighted interpolation: synthetic census tracts to neighborhoods
# ---------------------------------------------------------------------

ct_area_interp_gdf = tobler.area_weighted.area_interpolate(source_df=ct_synth_gdf, 
                                                    target_df=nbhd_synth_gdf,
                                                    extensive_variables=['ct_synth_pop'])

# Round the interpolation results to the nearest integer and change the type to integer
# (Population counts must be integers)
ct_area_interp_gdf['ct_synth_pop'] = ct_area_interp_gdf['ct_synth_pop'].round(decimals=0).astype(int)

# Rename the results column for clarity later
ct_area_interp_gdf = ct_area_interp_gdf.rename({'ct_synth_pop':'ct_area_interp_est'}, axis=1)

In [ ]:
# Area weighted interpolation: synthetic dissemination areas to neighborhoods
# ---------------------------------------------------------------------------

da_area_interp_gdf = tobler.area_weighted.area_interpolate(source_df=da_synth_gdf, 
                                                    target_df=nbhd_synth_gdf,
                                                    extensive_variables=['da_synth_pop'])

# Round the interpolation results to the nearest integer and change the type to integer
# (Population counts must be integers)
da_area_interp_gdf['da_synth_pop'] = da_area_interp_gdf['da_synth_pop'].round(decimals=0).astype(int)

# Rename the results column for clarity later
da_area_interp_gdf = da_area_interp_gdf.rename({'da_synth_pop':'da_area_interp_est'}, axis=1)

In [ ]:
# Dasymetric interpolation: synthetic census tracts + urban landover to neighborhoods
#------------------------------------------------------------------------------------

# Perform dasymetric interpolation
ct_dasy_interp_gdf = tobler.dasymetric.masked_area_interpolate(source_df=ct_synth_gdf, 
                                                        target_df=nbhd_synth_gdf,
                                                        raster='data/ottawa_landcover.tif',
                                                        codes=[17],
                                                        extensive_variables=['ct_synth_pop'])

# Round the interpolation results to the nearest integer and change the type to integer
# (Population counts must be integers)
ct_dasy_interp_gdf['ct_synth_pop'] = ct_dasy_interp_gdf['ct_synth_pop'].round(decimals=0).astype(int)

# Rename the results column for clarity later
ct_dasy_interp_gdf = ct_dasy_interp_gdf.rename({'ct_synth_pop':'ct_dasy_interp_est'}, axis=1)

# ~30 s ; ignore warnings

In [ ]:
# Dasymetric interpolation: synthetic dissemination areas + urban landover to neighborhoods
#------------------------------------------------------------------------------------------

# Perform dasymetric interpolation
da_dasy_interp_gdf = tobler.dasymetric.masked_area_interpolate(source_df=da_synth_gdf, 
                                                        target_df=nbhd_synth_gdf,
                                                        raster='data/ottawa_landcover.tif',
                                                        codes=[17],
                                                        extensive_variables=['da_synth_pop'])

# Round the interpolation results to the nearest integer and change the type to integer
# (Population counts must be integers)
da_dasy_interp_gdf['da_synth_pop'] = da_dasy_interp_gdf['da_synth_pop'].round(decimals=0).astype(int)

# Rename the results column for clarity later
da_dasy_interp_gdf = da_dasy_interp_gdf.rename({'da_synth_pop':'da_dasy_interp_est'}, axis=1)

# ~1.5 mins ; ignore warnings

In [ ]:
# Merge the results for comparison
#---------------------------------

# Create a list of the GeoDataFrames (drop the redundant geometry)
dfs = [nbhd_synth_gdf,
        ct_area_interp_gdf.drop(columns='geometry'),
        ct_dasy_interp_gdf.drop(columns='geometry'),
        da_area_interp_gdf.drop(columns='geometry'),
        da_dasy_interp_gdf.drop(columns='geometry'),]

# Concatenate the GeoDataFrames
interp_results_gdf = pd.concat(dfs, axis=1)

# Get into on the new interpolation results GeoDataFrame
interp_results_gdf.info()

In [ ]:
# Evaluate Results
# ----------------

ct_area_est = interp_results_gdf['ct_area_interp_est']
ct_dasy_est = interp_results_gdf['ct_dasy_interp_est']
da_area_est = interp_results_gdf['da_area_interp_est']
da_dasy_est = interp_results_gdf['da_dasy_interp_est']
expected = interp_results_gdf['nbhd_synth_pop']

# Percent Error - create new columns in interp_results_gdf
interp_results_gdf['ct_area_%_error'] = round(percent_error(ct_area_est, expected), 1)
interp_results_gdf['ct_dasy_%_error'] = round(percent_error(ct_dasy_est, expected), 1)
interp_results_gdf['da_area_%_error'] = round(percent_error(da_area_est, expected), 1)
interp_results_gdf['da_dasy_%_error'] = round(percent_error(da_dasy_est, expected), 1)

# Other statistics - create DataFrame of statistics
interp_stats_df = pd.DataFrame({'Interpolation Method': ['Area Weighted','Area Weighted',
                                                        'Dasymetric', 'Dasymetric'],
                                'Source Geographies': ['Census Tracts', 'Dissemination Areas',
                                                       'Census Tracts', 'Dissemination Areas'],
                                'RMSE': [round(rmse(ct_area_est, expected), 2),
                                        round(rmse(da_area_est, expected), 2),
                                        round(rmse(ct_dasy_est, expected), 2),
                                        round(rmse(da_dasy_est, expected), 2)],
                                'NRMSE': [round(nrmse(ct_area_est, expected), 2),
                                        round(nrmse(da_area_est, expected), 2),
                                        round(nrmse(ct_dasy_est, expected), 2),
                                        round(nrmse(da_dasy_est, expected), 2)],            
                                'MBE': [round(mbe(ct_area_est, expected), 2),
                                        round(mbe(da_area_est, expected), 2),
                                        round(mbe(ct_dasy_est, expected), 2),
                                        round(mbe(da_dasy_est, expected), 2)],
                                'MAE': [round(mae(ct_area_est, expected), 2),
                                        round(mae(da_area_est, expected), 2),
                                        round(mae(ct_dasy_est, expected), 2),
                                        round(mae(da_dasy_est, expected), 2)],
                                'r': [round(r_value(ct_area_est, expected),2 ),
                                        round(r_value(da_area_est, expected), 2),
                                        round(r_value(ct_dasy_est, expected),2 ),
                                        round(r_value(da_dasy_est, expected), 2)],
                                'r2': [round(r_value(ct_area_est, expected)**2, 2),
                                        round(r_value(da_area_est, expected)**2, 2),
                                        round(r_value(ct_dasy_est, expected)**2, 2),
                                        round(r_value(da_dasy_est, expected)**2, 2)]})

interp_stats_df

In [ ]:
# Plot the Percent Error of Both Methods using a Plotly Facet Map
# https://plotly.com/python/choropleth-maps/
# https://plotly.github.io/plotly.py-docs/generated/plotly.express.choropleth.html
# https://plotly.com/python/facet-plots/
# --------------------------------------------------------------------------------

# Convert interp_results_gdf from GeoDataFrame to GeoJson
geojson = interp_results_gdf.to_crs(epsg='4326').to_json()
geojson = json.loads(geojson)

# Reformat the interp_results_gdf for the plot
df = pd.DataFrame(interp_results_gdf.drop(columns='geometry'))

df = df.rename({'ct_area_%_error':'Source: Census Tracts <br> Method: Area Weighted',
                'ct_dasy_%_error':'Source: Census Tracts <br> Method: Dasymetric',
                'da_area_%_error':'Source: Dissemination Areas <br> Method: Area Weighted',
                'da_dasy_%_error':'Source: Dissemination Areas <br> Method: Dasymetric',
                }, axis=1)

df = df.melt(id_vars='nbhd_name',
            value_vars=['Source: Census Tracts <br> Method: Area Weighted',
                        'Source: Census Tracts <br> Method: Dasymetric',
                        'Source: Dissemination Areas <br> Method: Area Weighted',
                        'Source: Dissemination Areas <br> Method: Dasymetric'],
            var_name='method', 
            value_name='Error (%)')

# Create the Plotly Express choropleth figure
fig = px.choropleth(data_frame=df,
                    title="Areal Interpolation of Synthetic Population Data to Ottawa, ON Neighborhoods",
                    locations='nbhd_name',
                    geojson=geojson,
                    featureidkey='properties.nbhd_name',
                    color='Error (%)',
                    facet_col='method',
                    facet_col_wrap=2,
                    range_color=[0,100],
                    color_continuous_scale='Inferno',
                    projection='mercator',
                    fitbounds="locations",
                    height=800, width=800)
         
fig.for_each_annotation(lambda a: a.update(text=a.text.split('=')[-1]))
fig.show()

In [ ]:
# Create 1:1 Facet Plots using Plotly
# -------------------------------------
# https://plotly.com/python/line-and-scatter/
# https://plotly.github.io/plotly.py-docs/generated/plotly.express.scatter.html
# https://plotly.com/python/facet-plots/

# Reformat the interp_results_gdf for the plot
# --------------------------------------------

# Convert the interpolation results to a DataFrame
df = pd.DataFrame(interp_results_gdf.drop(columns='geometry'))

# Rename the results columns as they will become the facet plot labels
df = df.rename({'ct_area_interp_est':'Source: Census Tracts <br> Method: Area Weighted',
                'ct_dasy_interp_est':'Source: Census Tracts <br> Method: Dasymetric',
                'da_area_interp_est':'Source: Dissemination Areas <br> Method: Area Weighted',
                'da_dasy_interp_est':'Source: Dissemination Areas <br> Method: Dasymetric'},
                axis=1)

# Combine all the results columns into one column
df = df.melt(id_vars=['nbhd_name', 'nbhd_synth_pop'],
            value_vars=['Source: Census Tracts <br> Method: Area Weighted',
                        'Source: Census Tracts <br> Method: Dasymetric',
                        'Source: Dissemination Areas <br> Method: Area Weighted',
                        'Source: Dissemination Areas <br> Method: Dasymetric'],
            var_name='method', 
            value_name='interp_pop_est')

# Add a percent error column
estimated = df['interp_pop_est']
expected = df['nbhd_synth_pop']
df['%_error'] = round(percent_error(estimated, expected), 1)

# Create the Plotly Express Scatter figure
fig = px.scatter(data_frame=df,
                title="Areal Interpolation of Synthetic Population Data to Ottawa, ON Neighborhoods",
                x='nbhd_synth_pop',
                y='interp_pop_est',
                height=800, width=800,
                color='%_error',
                facet_col='method',
                facet_col_wrap=2,
                hover_name='nbhd_name',
                labels={'interp_pop_est': 'Interpolated Population',
                        'nbhd_synth_pop': ' Estimated Population'},
                color_continuous_scale='Inferno',
                range_color=[0,100])

# Create the 1:1 line
line_max = max([max(df['interp_pop_est']), max(df['nbhd_synth_pop'])])
line_min = min([min(df['interp_pop_est']), min(df['nbhd_synth_pop'])])

fig.update_layout(shapes=[
        dict(type='line', xref='x', yref='y',
            x0=line_min, y0=line_min, x1=line_max, y1=line_max, line_width=1),
        dict(type='line', xref='x2', yref='y2',
            x0=line_min, y0=line_min, x1=line_max, y1=line_max, line_width=1),
        dict(type='line', xref='x3', yref='y3',
            x0=line_min, y0=line_min, x1=line_max, y1=line_max, line_width=1),
        dict(type='line', xref='x4', yref='y4',
            x0=line_min, y0=line_min, x1=line_max, y1=line_max, line_width=1)])

fig.for_each_annotation(lambda a: a.update(text=a.text.split('=')[-1]))

fig.show()